In [1]:
from IPython.display import clear_output

!pip install -U accelerate
!pip install -U transformers
!pip install datasets
!pip install evaluate

clear_output()

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import load_dataset
import evaluate

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List
import string

import pandas as pd
import random
import os

import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
sns.set(palette='Paired')
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import re
import time
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('/content/drive/MyDrive/train (4).csv',sep = '\t')

In [4]:
df = df.drop('idx',axis = 1)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Score'] = le.fit_transform(df['Score'])

In [6]:
df

,Score,Text
0,1,В Альфа-Банке работает замечательная девушка -...
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,1,Очень порадовала оперативность работы в банке....
3,0,Имела неосторожность оформить потреб. кредит в...
4,0,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...
13997,1,Добрый день! Давно являюсь клиентом банка Русс...


In [7]:
df2 = pd.DataFrame()
df2['text'] = df['Text']
df2['label'] = df['Score']

In [8]:
df2

,text,label
0,В Альфа-Банке работает замечательная девушка -...,1
1,Оформляя рассрочку в м. Видео в меге тёплый ст...,0
2,Очень порадовала оперативность работы в банке....,1
3,Имела неосторожность оформить потреб. кредит в...,0
4,Небольшая предыстория: Нашел на сайте MDM банк...,0
...,...,...
13994,"О высокой надёжности МКБ, порядочности и добро...",1
13995,"Обслуживаюсь в офисе на Чернореченской 42а, ка...",1
13996,Попала сегодня в очень неприятную ситуацию. Ре...,1
13997,Добрый день! Давно являюсь клиентом банка Русс...,1


In [9]:
df2.to_csv('train.csv',index = False)

In [10]:
# датасет для бинарной классификации
full_dataset_1 = load_dataset('csv', data_files='/content/train.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
full_dataset_1 = full_dataset_1.filter(lambda example: example['text'] is not None and example['label'] is not None)

full_dataset_1

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/13999 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 13999
    })
})

In [11]:
dataset = full_dataset_1['train'].train_test_split(test_size=0.2, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11199
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2800
    })
})

In [12]:
dataset['train'][0]

{'text': 'Пытался получить ипотечный кредит в период резкого роста цен на недвижимость. По неопытности метался между банками, побывал в 5-ти банках Петербурга. Райффайзен мне показался самым удобным.Плюсы:- вежливый внимательный персонал (я общался с тремя девушками в кредитном отделе и на ресепшн. Остались самые лучшие впечатления).- грамотно составленные анкеты (заполнил за 5 минут. Все просто, удобно, разумно).- никаких очередей (не считая телефонных. Каждый раз, когда я заходил в отделение на Ефимова или в отделение на Мойке, я быстро получал то, что мне было нужно).- хорошо проработанные процедуры (в качестве примера - порадовал недавний звонок: Вы так и не воспользовались кредитом, что предпочитаете - забрать документы или чтобы мы их уничтожили?, ни в одном другом не такого не предлагали).- конкурентноспособные банковские продукты (одна из самых низких ставок на ипотечный кредит у нас в городе).Минусы:- название. Произнести я еще это могу, но написать... Ни по-русски, ни латиниц

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch

model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,ignore_mismatched_sizes=True)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

In [15]:
# токенизация текста
dataset = dataset.map(lambda e:
                      tokenizer(e['text'],
                                truncation = True,
                                max_length=512,
                                padding='max_length'), batched=True)

Map:   0%|          | 0/11199 [00:00<?, ? examples/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11199
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2800
    })
})

In [17]:
dataset = dataset.remove_columns('text')

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11199
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2800
    })
})

In [19]:
# переносим на гпу
dataset.set_format(type='torch')

In [20]:
dataset['train'][0]

{'label': tensor(1),
 'input_ids': tensor([    2, 44314,  4465, 18348, 32892,  8034, 35691,   314,  3915, 51813,
         29426, 30647,   548, 39136,    18,  1499, 62248,  3795, 21554,   938,
          2504, 55137,    16, 43800,   314,    25,    17,  9586, 47622, 17278,
            18, 34256, 48354,   775, 18759, 20284, 62638, 10206, 47630,    18,
         36622,   700,    30,    17, 42395, 41320, 80872,   775, 39149,    12,
           343, 52353,   329, 26268, 64534,   314, 35691,  2058, 40360,   320,
           548, 29823,  9438, 46430,    18, 32150,  4388, 29281, 32580, 36358,
            13,    18,    17, 46117, 46415,  1308, 64371,    12, 56033,   869,
           650,    25, 14860,    18,  9057,  9922,    16, 33565,    16, 43568,
            13,    18,    17, 30382, 70594,    12,   769, 36952, 55460,    18,
         30503,  4495,    16,  3570,   343, 74363,   314, 16103,   548, 71675,
           603,  1109,   314, 16103,   548, 33249,  1550,    16,   343, 15398,
         37936,  1

In [21]:
# тест модели
outputs = model(dataset['train']['input_ids'][0].unsqueeze(0).to(device), dataset['train']['attention_mask'][0].unsqueeze(0).to(device), dataset['train']['label'][0].unsqueeze(0).to(device))
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0672,  0.0152]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [55]:
from transformers import Trainer, TrainingArguments

# from datasets import load_metric
import evaluate

# Загрузка метрики вне функции
# metric_f1 = evaluate.load("accuracy")

# Параметры обучения
training_args = TrainingArguments(
    output_dir="cache",
    # learning_rate=3e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to = 'none'

)

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

trainer = Trainer(
    model=model,  # Модель для обучения
    args=training_args,  # Аргументы обучения
    train_dataset=dataset["train"],  # Набор данных для обучения
    eval_dataset=dataset["test"],  # Набор данных для валидации
    compute_metrics=compute_metrics,  # Метрики для вычисления
)

In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.305318,0.948929
2,No log,0.265622,0.952857
3,0.013900,0.283717,0.952143
4,0.013900,0.296016,0.953571
5,0.013900,0.296931,0.952500
6,0.003100,0.297781,0.952857


TrainOutput(global_step=1050, training_loss=0.008207951535781225, metrics={'train_runtime': 432.0483, 'train_samples_per_second': 155.524, 'train_steps_per_second': 2.43, 'total_flos': 495502463545344.0, 'train_loss': 0.008207951535781225, 'epoch': 6.0})

In [24]:
test = pd.read_csv('/content/drive/MyDrive/test (3).csv',sep='\t')

In [25]:
test = test.drop('idx',axis = 1)

In [26]:
test

,Text
0,Замечательная история об открытии счета в банк...
1,"23.07.14г. я досрочно внес сумму, указанную оп..."
2,Была задолженность по кредитной карте. Банк по...
3,Срочно понадобилась определенная денежная сумм...
4,Здравствуйте. В местном отделении Бинбанка г. ...
...,...
5995,Уважаемые представители Русфинанс Банка! Прошу...
5996,04 июня сняли с дебетовой карты банка 10 000 T...
5997,В апреле решил я купить шуруповерт. Посмотрел ...
5998,Недавно взял потребкредит в этом банке. Все че...


In [32]:
smp = pd.read_csv('/content/sample_submission (5).csv',sep='\t')

In [33]:
smp

,idx,Score
0,13999,Positive
1,14000,Positive
2,14001,Positive
3,14002,Positive
4,14003,Positive
...,...,...
5995,19994,Positive
5996,19995,Positive
5997,19996,Positive
5998,19997,Positive


In [34]:
test

,Text
0,Замечательная история об открытии счета в банк...
1,"23.07.14г. я досрочно внес сумму, указанную оп..."
2,Была задолженность по кредитной карте. Банк по...
3,Срочно понадобилась определенная денежная сумм...
4,Здравствуйте. В местном отделении Бинбанка г. ...
...,...
5995,Уважаемые представители Русфинанс Банка! Прошу...
5996,04 июня сняли с дебетовой карты банка 10 000 T...
5997,В апреле решил я купить шуруповерт. Посмотрел ...
5998,Недавно взял потребкредит в этом банке. Все че...


In [35]:
test.to_csv('test.csv',index = False)

In [39]:
full_dataset_2 = load_dataset('csv', data_files='/content/test.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
full_dataset_2 = full_dataset_2.filter(lambda example: example['Text'] is not None)

full_dataset_2

Filter:   0%|          | 0/6000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 6000
    })
})

In [57]:
pred_dataset = full_dataset_2['train']

pred_dataset = pred_dataset.map(lambda e: tokenizer(e['Text'],
                                truncation = True,
                                max_length=512,
                                padding='max_length'), batched=True)
# Remove the raw text column to avoid issues with the model
pred_dataset = pred_dataset.remove_columns("Text")

# Initialize the Trainer
trainer = Trainer(model=model)

# Make predictions
predictions = trainer.predict(pred_dataset)

# Get predicted class indices
predicted_class_indices = predictions.predictions.argmax(-1)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [58]:
predicted_class_indices

array([0, 0, 0, ..., 0, 1, 1])

In [59]:
smp['Score'] = predicted_class_indices

In [60]:
smp

,idx,Score
0,13999,0
1,14000,0
2,14001,0
3,14002,0
4,14003,0
...,...,...
5995,19994,1
5996,19995,1
5997,19996,0
5998,19997,1


In [61]:
smp['Score'] = le.inverse_transform(smp['Score'])

In [62]:
smp

,idx,Score
0,13999,Negative
1,14000,Negative
2,14001,Negative
3,14002,Negative
4,14003,Negative
...,...,...
5995,19994,Positive
5996,19995,Positive
5997,19996,Negative
5998,19997,Positive


In [64]:
smp.to_csv('otv4.csv',sep='\t',index=False)